In [3]:
import pandas as pd
from collections import defaultdict

In [4]:
from tika import parser

raw = parser.from_file('LIWC2015 dictionary poster.pdf')


2021-02-24 22:55:45,382 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-02-24 22:55:45,801 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-02-24 22:55:46,050 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [5]:
lines = raw['content'].split("\n")

In [94]:
cat_df = pd.read_csv("cat_df.csv")
cat_df = cat_df.drop(['Unnamed: 0','counter','col_count'],axis=1)

In [95]:
_sparse = []
cat_df['counter'] = cat_df['last_row']
cat_df['col_count'] = cat_df.apply(lambda row: row[2] if row[4]>0 else row[2]-1,axis=1)

In [96]:
for i in range(155):
    r = []
    for idx, row in cat_df.iterrows():
        diff = row['column']-row['col_count']
        
        if diff==0:
            r.extend([1]*row['column'])
        elif diff==1:
            r.extend([1]*row['col_count'])
            r.extend([0])
        elif diff == 0:
            r.extend([0]*row['column'])
        else:
            print(idx,row)
    _sparse.append(r)
    cat_df['counter'] = cat_df['counter']-1
    cat_df['col_count'] = cat_df.apply(lambda row: row[2] if row[4]>0 else row[2]-1,axis=1)

In [111]:
column_count = [sum(line) for line in _sparse]

In [100]:
#line_129 = '1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,0'.replace(" ","")
line_154 = '1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,1,1,0,1,0,1,0,0,1,0,0,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,0,0,1,1,0,1,1,0,0,1,1,1,1,1,1,0,1,1,0,1,1,0,1,0,1,1,0,1,0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0'

In [110]:
_sparse = _sparse[:154]+[[int(i)for i in line_154.split(",")]]

In [80]:
df2 = pd.read_csv('pdf_v2.csv', names=['text'])

In [79]:
#checkpoint
[len(row[0].split()) for idx, row in df2.iterrows()]==column_count

True

In [120]:
cat_df['counter'] = cat_df['last_row']
cat_df['col_count'] = cat_df.apply(lambda row: row[2] if row[4]>0 else row[2]-1,axis=1)
cat_df

,cat_num,cat_name,column,last_row,counter,col_count
0,1,Function,4,26,26,4
1,2,Pronoun,1,153,153,1
2,3,Ppron,1,93,93,1
3,4,I,1,24,24,1
4,5,We,1,12,12,1
...,...,...,...,...,...,...
68,121,Swear,1,131,131,1
69,122,Netspeak,2,62,62,2
70,123,Assent,1,36,36,1
71,124,Nonflu,1,19,19,1


In [113]:
df2.head(3)

,text
0,a however so would've another he i let's ily* ...
1,about howre somebod* wouldn't anybod* he'd i'd...
2,above hows somehow wouldnt anymore he's i'd've...


In [121]:
#_sparse
d = defaultdict(list) 

for idx, row in df2.iterrows():
    column_num = 0
    film = _sparse[idx]
    line = row['text'].split()
    print("checkpoint row {}: {}".format(idx,sum(film)==len(line)))

    for idx2, row2 in cat_df.iterrows():
        diff = row2['column']-row2['col_count']
        if idx==154 and idx2==68:
            for i in line[column_num:column_num+6]:
                d[i].append(row2['cat_num'])
            column_num = column_num+6
        elif diff==0:
            for i in line[column_num:column_num+row2['column']]:
                d[i].append(row2['cat_num'])
            column_num = column_num+row2['column']
        elif diff==1:
            for i in line[column_num:(column_num+row2['column']-1)]:
                d[i].append(row2['cat_num'])
            column_num = column_num+row2['column']-1

    cat_df['counter'] = cat_df['counter']-1
    cat_df['col_count'] = cat_df.apply(lambda row: row[2] if row[4]>0 else row[2]-1,axis=1)
    

checkpoint row 0: True
checkpoint row 1: True
checkpoint row 2: True
checkpoint row 3: True
checkpoint row 4: True
checkpoint row 5: True
checkpoint row 6: True
checkpoint row 7: True
checkpoint row 8: True
checkpoint row 9: True
checkpoint row 10: True
checkpoint row 11: True
checkpoint row 12: True
checkpoint row 13: True
checkpoint row 14: True
checkpoint row 15: True
checkpoint row 16: True
checkpoint row 17: True
checkpoint row 18: True
checkpoint row 19: True
checkpoint row 20: True
checkpoint row 21: True
checkpoint row 22: True
checkpoint row 23: True
checkpoint row 24: True
checkpoint row 25: True
checkpoint row 26: True
checkpoint row 27: True
checkpoint row 28: True
checkpoint row 29: True
checkpoint row 30: True
checkpoint row 31: True
checkpoint row 32: True
checkpoint row 33: True
checkpoint row 34: True
checkpoint row 35: True
checkpoint row 36: True
checkpoint row 37: True
checkpoint row 38: True
checkpoint row 39: True
checkpoint row 40: True
checkpoint row 41: True
ch

In [124]:
d['pinterest'].append(122)
d['pls'].append(122)

In [128]:
import csv
with open('dict.csv', 'w') as f:
    for key in d.keys():
        f.write("%s,%s\n"%(key,d[key]))

In [129]:
df_dict = pd.DataFrame.from_dict(d, orient='index')

In [131]:
df_dict.to_csv('dict.csv')

In [132]:
df_dict.head(3)

,0,1,2,3,4,5,6,7,8,9
a,1,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
however,1,14.0,50.0,56.0,13.0,NaN,NaN,NaN,NaN,NaN
so,1,14.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:

import liwc
parse, category_names = liwc.load_token_parser('queryDictionary.dic')

In [2]:
news_df = pd.read_csv("news_data.csv")

NameError: name 'pd' is not defined

In [5]:
import re
from collections import Counter

def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

gettysburg = '''absolute all always clear'''

gettysburg_tokens = tokenize(gettysburg)
# now flatmap over all the categories in all of the tokens using a generator:
gettysburg_counts = Counter(category for token in gettysburg_tokens for category in parse(token))
# and print the results:
print(gettysburg_counts)

Counter({'Certain': 4, 'CogProc': 4, 'Quant': 1, 'Time': 1, 'Relativ': 1, 'See': 1, 'Percept': 1, 'Adj': 1})


In [115]:
def parseLIWC(x):
    gettysburg_tokens = tokenize(x)
    gettysburg_counts = Counter(category for token in gettysburg_tokens for category in parse(token))
    return gettysburg_counts

In [116]:
news_df['title_liwc_ct'] = news_df['title'].apply(parseLIWC)

In [1]:
import json
json_struct = json.loads(news_df.to_json(orient="records"))    
df_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json

NameError: name 'news_df' is not defined

In [129]:
df_flat = df_flat.fillna(0)
df_flat['total'] = df_flat.sum(axis=1)

In [124]:
df_flat = df_flat.drop(['Unnamed: 0','file_name', 'news_content','title'], axis=1)
df_flat.fillna(0)

In [133]:
df_flat

,category,folder,title_liwc_ct.Social,title_liwc_ct.FocusPresent,title_liwc_ct.Function,title_liwc_ct.Verb,title_liwc_ct.Auxverb,title_liwc_ct.Ipron,title_liwc_ct.Article,title_liwc_ct.Swear,...,title_liwc_ct.Male,title_liwc_ct.Affiliation,title_liwc_ct.Body,title_liwc_ct.Home,title_liwc_ct.Leisure,title_liwc_ct.We,title_liwc_ct.They,title_liwc_ct.Relig,title_liwc_ct.Risk,total
0,Buzzfeed Political News Dataset,Real,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
1,Buzzfeed Political News Dataset,Real,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,Buzzfeed Political News Dataset,Real,0.0,0.0,1.0,1.0,3.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
3,Buzzfeed Political News Dataset,Real,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Buzzfeed Political News Dataset,Real,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Buzzfeed Political News Dataset,Real,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0
6,Buzzfeed Political News Dataset,Real,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
7,Buzzfeed Political News Dataset,Real,0.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
8,Buzzfeed Political News Dataset,Real,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0
9,Buzzfeed Political News Dataset,Real,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [131]:
df_flat.groupby(['category','folder']).agg(['sum'])

title_liwc_ct.Social  \
                                                        sum   
category                        folder                        
Buzzfeed Political News Dataset Fake                    2.0   
                                Real                    4.0   
Random Poltical News Dataset    Fake                    0.0   
                                Real                   14.0   
                                Satire                  3.0   

                                       title_liwc_ct.FocusPresent  \
                                                              sum   
category                        folder                              
Buzzfeed Political News Dataset Fake                          9.0   
                                Real                         25.0   
Random Poltical News Dataset    Fake                         14.0   
                                Real                         27.0   
                                Satire                        8.0   

                                       title_liwc_ct.Function  \
                                                          sum   
category                        folder                          
Buzzfeed Political News Dataset Fake                     17.0   
                                Real                     48.0   
Random Poltical News Dataset    Fake                     26.0   
                                Real                     44.0   
                                Satire                   11.0   

                                       title_liwc_ct.Verb  \
                                                      sum   
category                        folder                      
Buzzfeed Political News Dataset Fake                  6.0   
                                Real                 24.0   
Random Poltical News Dataset    Fake                  5.0   
                                Real                 10.0   
                                Satire                3.0   

                                       title_liwc_ct.Auxverb  \
                                                         sum   
category                        folder                         
Buzzfeed Political News Dataset Fake                     5.0   
                                Real                    19.0   
Random Poltical News Dataset    Fake                     5.0   
                                Real                     8.0   
                                Satire                   1.0   

                                       title_liwc_ct.Ipron  \
                                                       sum   
category                        folder                       
Buzzfeed Political News Dataset Fake                   0.0   
                                Real                   3.0   
Random Poltical News Dataset    Fake                   2.0   
                                Real                   7.0   
                                Satire                 3.0   

                                       title_liwc_ct.Article  \
                                                         sum   
category                        folder                         
Buzzfeed Political News Dataset Fake                     7.0   
                                Real                    12.0   
Random Poltical News Dataset    Fake                     8.0   
                                Real                     9.0   
                                Satire                   3.0   

                                       title_liwc_ct.Swear  \
                                                       sum   
category                        folder                       
Buzzfeed Political News Dataset Fake                   0.0   
                                Real                   1.0   
Random Poltical News Dataset    Fake                   0.0   
                                Real                   0.0   
              

In [105]:
pd.set_option('display.max_rows',news_df.shape[0])